# Homework 1

### Setting up

In [1]:
import pandas as pd
import numpy as np

Check pandas version

In [2]:
pd.__version__

'2.2.2'

import the dataset ***laptops.csv***

In [3]:
laptops = pd.read_csv("../../Datasets/laptops/laptops.csv")

In [4]:
laptops.head()

,Laptop,Status,Brand,Model,CPU,RAM,Storage,Storage type,GPU,Screen,Touch,Final Price
0,ASUS ExpertBook B1 B1502CBA-EJ0436X Intel Core...,New,Asus,ExpertBook,Intel Core i5,8,512,SSD,NaN,15.6,No,1009.00
1,Alurin Go Start Intel Celeron N4020/8GB/256GB ...,New,Alurin,Go,Intel Celeron,8,256,SSD,NaN,15.6,No,299.00
2,ASUS ExpertBook B1 B1502CBA-EJ0424X Intel Core...,New,Asus,ExpertBook,Intel Core i3,8,256,SSD,NaN,15.6,No,789.00
3,MSI Katana GF66 12UC-082XES Intel Core i7-1270...,New,MSI,Katana,Intel Core i7,16,1000,SSD,RTX 3050,15.6,No,1199.00
4,HP 15S-FQ5085NS Intel Core i5-1235U/16GB/512GB...,New,HP,15S,Intel Core i5,16,512,SSD,NaN,15.6,No,669.01


### Homework

In [5]:
laptops.count()

Laptop          2160
Status          2160
Brand           2160
Model           2160
CPU             2160
RAM             2160
Storage         2160
Storage type    2118
GPU              789
Screen          2156
Touch           2160
Final Price     2160
dtype: int64

I want to confirm the number of values in *GPU*.\
The code below is how I thought of doing it.

In [6]:
# count number of null values and add to the count of values
laptops['GPU'].isnull().sum() + laptops['GPU'].count()

2160

To find the laptop brands. I think ***.unique()*** function should work

In [7]:
# features list
features = laptops.columns

I decided to create a list of features so that I won't have to type the string every single time. I just need to type in the index


In [8]:
laptops[features[2]].unique()

array(['Asus', 'Alurin', 'MSI', 'HP', 'Lenovo', 'Medion', 'Acer', 'Apple',
       'Razer', 'Gigabyte', 'Dell', 'LG', 'Samsung', 'PcCom', 'Microsoft',
       'Primux', 'Prixton', 'Dynabook Toshiba', 'Thomson', 'Denver',
       'Deep Gaming', 'Vant', 'Innjoo', 'Jetwing', 'Millenium', 'Realme',
       'Toshiba'], dtype=object)

It seems that ***.unique()*** only returns the array of unique values. Since it is an array, we can probably use the ***.size*** method

In [9]:
laptops[features[2]].unique().size

27

Wait, I want to make sure that there are no unique values but somehow the strings doesn't match because of casing.\
For that, i can try using ***.lower()*** first before counting the unique values.

In [10]:
laptops[features[2]].str.lower().unique().size

27

Cool. So there are only **27** brands in this DataFrame.

Now we are asked to find how many columns have missing values. I can definitely use ***.isnull()*** for this.\
From the looks of it, there's only 1 column with missing values. I'm gonna have to confirm it first.

In [11]:
# laptops.isnull().sum() > 0
laptops.isnull().sum()

Laptop             0
Status             0
Brand              0
Model              0
CPU                0
RAM                0
Storage            0
Storage type      42
GPU             1371
Screen             4
Touch              0
Final Price        0
dtype: int64

Turns out there's three.

How about finding the maximum final price of Dell notebooks in the dataset?\
I think some filtering is needed for this. It goes like:
```
SELECT
    max('Screen')
FROM
    laptops
WHERE
    BRAND='DELL'
```

In [24]:
laptops[laptops['Brand'] == 'Dell'].Model.unique()

array(['Vostro', 'Latitude', 'XPS', 'Precision'], dtype=object)

Ok, that works but it's not enough to get our answer since we need the maximum final price.

In [26]:
# since our code above returns another dataframe, we can index it again
# then add .max() to get the maximum value in it.
max(laptops[laptops['Brand'] == 'Dell']['Final Price'].values)

3936.0

Next, we have to do the following:
```
1. Find the median value of Screen column in the dataset.
2. Next, calculate the most frequent value of the same Screen column.
3. Use fillna method to fill the missing values in Screen column with the most frequent value from the previous step.
4. Now, calculate the median value of Screen once again.
```

```
1. Find the median value of Screen column in the dataset.
```
To do this, maybe there is a median() function?

In [14]:
laptops['Screen'].median()

15.6

For #2, maybe there is also a mode() function?

In [15]:
print(laptops['Screen'].mode())
print(laptops['Screen'].mode().size)

0    15.6
Name: Screen, dtype: float64
1


Weird, why does it show "0 15.6"?

In [16]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mode.html
# after checking it, seems like 0 is an axis?
# I wanna try something
print(laptops.mode(axis='columns', numeric_only=True).head(1))

     0     1      2       3
0  8.0  15.6  512.0  1009.0


I see! so it is an axis.

For #3, I want to first see what fillna does

In [17]:
help(laptops.fillna)

Help on method fillna in module pandas.core.generic:

fillna(value: 'Hashable | Mapping | Series | DataFrame | None' = None, *, method: 'FillnaOptions | None' = None, axis: 'Axis | None' = None, inplace: 'bool_t' = False, limit: 'int | None' = None, downcast: 'dict | None | lib.NoDefault' = <no_default>) -> 'Self | None' method of pandas.core.frame.DataFrame instance
    Fill NA/NaN values using the specified method.
    
    Parameters
    ----------
    value : scalar, dict, Series, or DataFrame
        Value to use to fill holes (e.g. 0), alternately a
        dict/Series/DataFrame of values specifying which value to use for
        each index (for a Series) or column (for a DataFrame).  Values not
        in the dict/Series/DataFrame will not be filled. This value cannot
        be a list.
    method : {'backfill', 'bfill', 'ffill', None}, default None
        Method to use for filling holes in reindexed Series:
    
        * ffill: propagate last valid observation forward to next

What would happen if I pass the result of .mode() as a parameter?

In [18]:
# get first element of mode
mode = laptops['Screen'].mode()[0]
# fill up null values and overwrite laptops
laptops['Screen'] = laptops['Screen'].fillna(mode)

In [19]:
laptops.count()

Laptop          2160
Status          2160
Brand           2160
Model           2160
CPU             2160
RAM             2160
Storage         2160
Storage type    2118
GPU              789
Screen          2160
Touch           2160
Final Price     2160
dtype: int64

Seems like it worked but what I need is to see if the median changed after that operation.\
I can just add ***.median()*** again to that line of code

In [20]:
laptops['Screen'].median()

15.6

It didn't change!

Alright, for the last task, we have this:
```
1. Select all the "Innjoo" laptops from the dataset.
2. Select only columns RAM, Storage, Screen.
3. Get the underlying NumPy array. Let's call it X.
4. Compute matrix-matrix multiplication between the transpose of X and X. To get the transpose, use X.T. Let's call the result XTX.
5. Compute the inverse of XTX.
6. Create an array y with values [1100, 1300, 800, 900, 1000, 1100].
7. Multiply the inverse of XTX with the transpose of X, and then multiply the result by y. Call the result w.
8. What's the sum of all the elements of the result?
```

In [21]:
## select all the "Innjoo" laptops from the dataset.
innjoo_dataset = laptops[laptops['Brand'] == "Innjoo"]

## select only columns RAM, Storage
features = ['RAM', 'Storage', 'Screen']

## get the underlying NumPy array and call it X
X = innjoo_dataset[features].values

## matrix-matrix multiplication between the transpose of X and X
# assign transpose to XT
XT = X.T
# multiply matrices and assign to XTX
XTX = XT.dot(X)

## compute for the inverse of XTX
XTX_inv = np.linalg.inv(XTX)

## create an array y with values [1100, 1300, 800, 900, 1000, 1100]
y = [1100, 1300, 800, 900, 1000, 1100]

## multiply the inverse of XTX with the transpose of X
UV = XTX_inv.dot(XT)
    
## and then multiply the result by y. Assign to 'w'
w = UV.dot(y)

# display sum of w rounded to 2 with 2 decimal places
"{:.2f}".format(w.sum().round(2))

'91.30'